Prediction of lizon user clones from best models created in lizon_transformers_final.ipynb, averaging by all models, finding clones and quick review of user-clones.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  1 16:38:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [5]:
import os
import time

import pandas as pd
import numpy as np

In [ ]:
!pip install ktrain

In [7]:
import ktrain
from ktrain import text

In [8]:
Data = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Data/'

test_Messages_filename='lizon_clone_data_for_test.csv'
test_Messages_full_filename= os.path.join(Data, test_Messages_filename)

Authors_filename='Authors.csv'
Authors_full_filename= os.path.join(Data, Authors_filename)

text_column='original_message'
classname='lizon'
prediction_column='prediction'

Models = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Models/blinoff256/'

threshold_lizon_probability=0.9
threshold_frc_lizon_messages=0.7

## Data for prediction

In [9]:
df=pd.read_csv(test_Messages_full_filename, error_bad_lines=False, index_col=False)

In [10]:
len(df)

123724

In [11]:
df.head()

,Message_Id,Timestamp,Parent_Id,Author_Id,Topic_Id,Topic,original_message,prediction_0,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8
0,81932578,2013-09-01 22:40:00,0,169074,1998529,А сколько у вас игрушек и каких?,"<div class=""body"">6,5 лет мальчик <br/>контейн...",0.000446,0.000539,0.000491,0.000534,0.000563,0.000403,0.000683,0.000539,0.000550
1,91092877,2015-11-16 09:18:00,0,312157,2302564,3 года и памперс,"<div class=""body"">Вот интересно, какая такая н...",0.000365,0.000443,0.000481,0.000464,0.000405,0.000310,0.000760,0.000415,0.000525
2,91098204,2015-11-16 19:02:00,91094634,312157,2302564,3 года и памперс,"<div class=""body"">Я уверена, если снять пампер...",0.000949,0.001422,0.000585,0.000892,0.001695,0.001870,0.005292,0.000545,0.001228
3,91098228,2015-11-16 19:06:00,91096034,312157,2302564,3 года и памперс,"<div class=""body"">Ну если ходит днем- этот топ...",0.007469,0.824557,0.047888,0.259741,0.198313,0.029584,0.283927,0.002375,0.137680
4,91101933,2015-11-17 08:36:00,91101707,312157,2302564,3 года и памперс,"<div class=""body"">Это все решается. У меня нет...",0.001513,0.001165,0.001101,0.000696,0.000707,0.000645,0.004604,0.001165,0.001184


## Prediction clone or not

In [15]:
prediction_column_lists=list()
ts_start = time.time()
for m in next(os.walk(Models))[1]:  
  ind= m.split('_')[len(m.split('_'))-1]
  prediction_column_lists.append('%s_%s'%(prediction_column,ind))
  #the notebook may be interruoted due to the lack of memory and can be restarted several times to predict from all models
  if '%s_%s'%(prediction_column,ind) in df.columns:
    print('%s model is already processed'%m)
    continue
  print('Processing %s model'%m)
  Model_foldername = m
  Model_full_foldername = os.path.join(Models, Model_foldername)

  predictor = ktrain.load_predictor(Model_full_foldername)

  prediction=predictor.predict_proba(df[text_column].tolist())

  df_prediction = pd.DataFrame(prediction, columns = predictor.get_classes()) 
  df_prediction=df_prediction[[classname]]
  df_prediction.columns=['%s_%s'%(prediction_column,ind)]

  df['%s_%s'%(prediction_column,ind)]=df_prediction['%s_%s'%(prediction_column,ind)].to_numpy()
  df.to_csv(test_Messages_full_filename, header=True, index=False)
  
  

ts_end = time.time()
print((ts_end - ts_start)/60)

fit_onecycle_0 model is already processed
fit_onecycle_1 model is already processed
fit_onecycle_2 model is already processed
fit_onecycle_3 model is already processed
fit_onecycle_4 model is already processed
fit_onecycle_5 model is already processed
fit_onecycle_6 model is already processed
fit_onecycle_7 model is already processed
fit_onecycle_8 model is already processed
Processing fit_onecycle_9 model


Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

18.36239205201467


## Average by all models/folds

In [19]:
df[prediction_column]=df[prediction_column_lists].mean(axis=1)

In [20]:
df.dtypes

Message_Id            int64
Timestamp            object
Parent_Id             int64
Author_Id             int64
Topic_Id              int64
Topic                object
original_message     object
prediction_0        float64
prediction_1        float64
prediction_2        float64
prediction_3        float64
prediction_4        float64
prediction_5        float64
prediction_6        float64
prediction_7        float64
prediction_8        float64
prediction_9        float32
prediction          float64
dtype: object

## Fraction of messages which looks like "lizon" messages in each author in the dataset

In [21]:
author_totals=df.groupby(['Author_Id']).size().reset_index(name='total_counts').sort_values(['total_counts'],ascending=False)
author_predicted=df[df[prediction_column]>threshold_lizon_probability].groupby(['Author_Id']).size().reset_index(name='predicted_counts').sort_values(['predicted_counts'],ascending=False)
author_predicted=pd.merge(author_totals,author_predicted, left_on='Author_Id', right_on='Author_Id', how='inner')

In [39]:
#to get one author name per Author_Id (then can vary in messages)
Authors=pd.read_csv(Authors_full_filename, error_bad_lines=False, index_col=False)

In [23]:
author_predicted=pd.merge(Authors[['Author_Id','author']],author_predicted, left_on='Author_Id', right_on='Author_Id', how='inner')

In [24]:
author_predicted['fraction_%s_messages'%classname]=author_predicted['predicted_counts']/author_predicted['total_counts']

In [25]:
author_predicted.sort_values(['fraction_%s_messages'%classname],ascending=False, inplace=True)

## lizon clones

In [26]:
author_predicted[author_predicted['fraction_lizon_messages']>threshold_frc_lizon_messages  ]

,Author_Id,author,total_counts,predicted_counts,fraction_lizon_messages
100,711697,Lizonn +,79,75,0.949367
104,731728,newyorck D,1045,965,0.923445
86,475085,Элиззи H*,2054,1858,0.904576
85,472696,Кассиапея D,635,566,0.891339
11,100899,lizlizon *,322,257,0.798137
46,300514,"Lizon "" **K**",555,393,0.708108


Let's check the rest of the authors with smaller score

In [29]:
author_predicted.head(10)

,Author_Id,author,total_counts,predicted_counts,fraction_lizon_messages
100,711697,Lizonn +,79,75,0.949367
104,731728,newyorck D,1045,965,0.923445
86,475085,Элиззи H*,2054,1858,0.904576
85,472696,Кассиапея D,635,566,0.891339
11,100899,lizlizon *,322,257,0.798137
46,300514,"Lizon "" **K**",555,393,0.708108
81,424711,Joconda Mona Liza _,227,154,0.678414
98,560177,ok591403023403 F,67,13,0.194030
0,44753,Ась H*,302,46,0.152318
32,214089,альтамира SD*,66,10,0.151515


Looks like Joconda Mona Liza _ is also a clone but with less score. All other authors have significantly smaller score

In [30]:
threshold_frc_lizon_messages=0.6

In [31]:
author_predicted[author_predicted['fraction_lizon_messages']>threshold_frc_lizon_messages  ]

,Author_Id,author,total_counts,predicted_counts,fraction_lizon_messages
100,711697,Lizonn +,79,75,0.949367
104,731728,newyorck D,1045,965,0.923445
86,475085,Элиззи H*,2054,1858,0.904576
85,472696,Кассиапея D,635,566,0.891339
11,100899,lizlizon *,322,257,0.798137
46,300514,"Lizon "" **K**",555,393,0.708108
81,424711,Joconda Mona Liza _,227,154,0.678414


In [32]:
df.to_csv(test_Messages_full_filename, header=True, index=False)

## All lizon users

In [33]:
lizon = author_predicted[author_predicted['fraction_lizon_messages']>threshold_frc_lizon_messages  ].copy(deep=True)

In [41]:
first_lizon_id = 69715

In [42]:
Authors[((Authors['Author_Id']==first_lizon_id) | (Authors['Author_Id'].isin(lizon['Author_Id'].tolist())))][['Author_Id','author', 'cnt_messages', 'cnt_topics_started','first_message_ts','last_message_ts','avg_immediate_childs', 'avg_childs', 'avg_messages_per_day', 'DayPartPeriodicity']].sort_values('first_message_ts',ascending=True)

,Author_Id,author,cnt_messages,cnt_topics_started,first_message_ts,last_message_ts,avg_immediate_childs,avg_childs,avg_messages_per_day,DayPartPeriodicity
3485,69715,lizon **,1274,19,2009-08-28 11:50:00,2010-12-07 23:18:00,1.061224,4.493721,0.356223,111
14771,300514,"Lizon "" **K**",955,30,2011-01-04 14:22:00,2013-01-19 20:38:00,1.062827,4.025131,0.288204,111
5399,100899,lizlizon *,471,11,2012-12-09 13:15:00,2013-12-27 03:05:00,1.108280,3.605096,0.246073,111
16940,424711,Joconda Mona Liza _,390,4,2014-02-07 02:23:00,2015-06-04 17:02:00,1.061538,4.782051,0.165975,111
17566,472696,Кассиапея D,1093,15,2015-06-24 22:40:00,2019-03-16 15:05:00,1.116194,4.318390,0.145588,111
17627,475085,Элиззи H*,3317,44,2015-07-26 22:11:00,2019-03-21 21:20:00,1.109135,4.608683,0.423106,111
19218,711697,Lizonn +,136,2,2018-12-29 16:55:00,2019-03-28 23:41:00,1.139706,3.632353,0.224719,111
19739,731728,newyorck D,2142,32,2020-02-17 15:43:00,2021-09-16 20:49:00,1.035481,3.554622,0.521664,111


In [49]:
print('Total number of lizon`s posts is %s'%Authors[((Authors['Author_Id']==first_lizon_id) | (Authors['Author_Id'].isin(lizon['Author_Id'].tolist())))]['cnt_messages'].sum())

Total number of lizon`s posts is 9778


In [50]:
print('Total number of topics lizon started is %s'%Authors[((Authors['Author_Id']==first_lizon_id) | (Authors['Author_Id'].isin(lizon['Author_Id'].tolist())))]['cnt_topics_started'].sum())

Total number of topics lizon started is 157


They all quite active users with a lot of posts and new topics started. Each has a lot of feedback (responses to their posts).
Some users exists at the same time (intersection between first and last messages ranges). Sometimes there are small gaps between activity on the forum. *italicized text*

In [48]:
tmp_df=df[df['Author_Id'].isin(lizon['Author_Id'].tolist())][['Topic_Id','Author_Id']].groupby(['Topic_Id']).nunique().reset_index().sort_values('Author_Id', ascending=False)
tmp_df.columns=['Topic_Id','cnt_lizon_authors']
tmp_df=tmp_df[tmp_df['cnt_lizon_authors']>1]
print('Number of topics where lizon is talking to herself using different users is %s'%len(tmp_df))

Number of topics where lizon is talking to herself using different users is 23


*Need full dataset to check if she replies to her own messages. df is only named users (no anonymous) and intermediate replies can be skipped, so the chain of a conversation may have gaps.*

In [52]:
Authors[((Authors['Author_Id']==first_lizon_id) | (Authors['Author_Id'].isin(lizon['Author_Id'].tolist())))][['Author_Id','author','first_message_ts','fraction_emojis','fraction_images','fraction_links','avg_message_words','avg_original_paragraphs','avg_actual_paragraphs','avg_paragraph_words']].sort_values('first_message_ts',ascending=True)

,Author_Id,author,first_message_ts,fraction_emojis,fraction_images,fraction_links,avg_message_words,avg_original_paragraphs,avg_actual_paragraphs,avg_paragraph_words
3485,69715,lizon **,2009-08-28 11:50:00,0.0,0.000000,0.142072,33.148352,1.276295,1.218210,5.774562
14771,300514,"Lizon "" **K**",2011-01-04 14:22:00,0.0,0.058639,0.127749,40.590576,1.413613,1.317277,7.457669
5399,100899,lizlizon *,2012-12-09 13:15:00,0.0,0.084926,0.203822,61.804671,3.477707,3.121019,10.526023
16940,424711,Joconda Mona Liza _,2014-02-07 02:23:00,0.0,0.115385,0.228205,42.987179,1.974359,1.671795,8.525708
17566,472696,Кассиапея D,2015-06-24 22:40:00,0.0,0.049405,0.097896,50.613907,2.793230,2.636780,8.819883
17627,475085,Элиззи H*,2015-07-26 22:11:00,0.0,0.141996,0.306602,61.345191,4.270425,3.731384,10.065658
19218,711697,Lizonn +,2018-12-29 16:55:00,0.0,0.492647,0.632353,48.970588,4.551471,3.426471,8.388504
19739,731728,newyorck D,2020-02-17 15:43:00,0.0,0.229225,0.301120,41.502334,3.131186,2.611111,7.331484


*She does not use emojies, but images and links to photos. That's the reason why using original, not clean text in the model is more successfull then cleaned, without html tags text.*

In [54]:
Authors[((Authors['Author_Id']==first_lizon_id) | (Authors['Author_Id'].isin(lizon['Author_Id'].tolist())))][['Author_Id','author','first_message_ts','cnt_messages','avg_immediate_childs','fraction_of_tatyana_neg_rersponses','fraction_of_tatyana_pos_rersponses']].sort_values('first_message_ts',ascending=True)

,Author_Id,author,first_message_ts,cnt_messages,avg_immediate_childs,fraction_of_tatyana_neg_rersponses,fraction_of_tatyana_pos_rersponses
3485,69715,lizon **,2009-08-28 11:50:00,1274,1.061224,0.437206,0.234694
14771,300514,"Lizon "" **K**",2011-01-04 14:22:00,955,1.062827,0.413613,0.268063
5399,100899,lizlizon *,2012-12-09 13:15:00,471,1.108280,0.469214,0.290870
16940,424711,Joconda Mona Liza _,2014-02-07 02:23:00,390,1.061538,0.479487,0.207692
17566,472696,Кассиапея D,2015-06-24 22:40:00,1093,1.116194,0.456542,0.273559
17627,475085,Элиззи H*,2015-07-26 22:11:00,3317,1.109135,0.489599,0.227314
19218,711697,Lizonn +,2018-12-29 16:55:00,136,1.139706,0.463235,0.235294
19739,731728,newyorck D,2020-02-17 15:43:00,2142,1.035481,0.448179,0.198413


*Responses to her messages are rather negative then positive. But, in fact, there are even more negative responses hided behind "sarcasm". Unfortunately, there are no pre-traned models to detect sarcasm in Russian langauge.*

In [55]:
Authors[((Authors['Author_Id']==first_lizon_id) | (Authors['Author_Id'].isin(lizon['Author_Id'].tolist())))][['Author_Id','author','first_message_ts','cnt_messages','fraction_of_tatyana_neg_messages','fraction_of_tatyana_pos_messages']].sort_values('first_message_ts',ascending=True)

,Author_Id,author,first_message_ts,cnt_messages,fraction_of_tatyana_neg_messages,fraction_of_tatyana_pos_messages
3485,69715,lizon **,2009-08-28 11:50:00,1274,0.460754,0.167190
14771,300514,"Lizon "" **K**",2011-01-04 14:22:00,955,0.450262,0.208377
5399,100899,lizlizon *,2012-12-09 13:15:00,471,0.486200,0.188960
16940,424711,Joconda Mona Liza _,2014-02-07 02:23:00,390,0.430769,0.197436
17566,472696,Кассиапея D,2015-06-24 22:40:00,1093,0.431839,0.203111
17627,475085,Элиззи H*,2015-07-26 22:11:00,3317,0.477841,0.184806
19218,711697,Lizonn +,2018-12-29 16:55:00,136,0.382353,0.191176
19739,731728,newyorck D,2020-02-17 15:43:00,2142,0.418301,0.188609


*But her own messages are not very positive as well.*